In [42]:
import os
import glob
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow import keras 
import tensorflow as tf

In [43]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

10 Physical GPUs, 10 Logical GPUs


In [44]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 10000 ):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables 
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load a subset of the data to memory 
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #separate into training and testing 
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names

In [45]:
x_train, y_train, x_test, y_test, class_names = load_data("./data")
print(class_names)
image_size = 28
num_classes = len(class_names)

# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

['duck', 'snail', 'banana', 'broccoli', 'carrot', 'bird', 'fish', 'watermelon', 'cloud', 'apple', 'blackberry', 'pig', 'cow', 'bush', 'leaf', 'flamingo', 'zebra', 'octopus', 'ant', 'grass', 'donut', 'kangaroo', 'blueberry', 'parrot', 'raccoon', 'camel', 'frog', 'spider', 'pear', 'crab', 'garden', 'sandwich', 'asparagus', 'flower', 'mouse', 'swan', 'steak', 'potato', 'crocodile', 'owl', 'horse', 'strawberry', 'grapes', 'pineapple', 'cake', 'bread']


In [46]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(46, activation='softmax')) 
# Train model
adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 576)              

In [47]:
#fit the model 
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

Train on 331200 samples, validate on 36800 samples
Epoch 1/5
331200/331200 - 5s - loss: 1.6175 - top_k_categorical_accuracy: 0.8284 - val_loss: 1.1974 - val_top_k_categorical_accuracy: 0.8997
Epoch 2/5
331200/331200 - 4s - loss: 1.1028 - top_k_categorical_accuracy: 0.9109 - val_loss: 1.0481 - val_top_k_categorical_accuracy: 0.9171
Epoch 3/5
331200/331200 - 4s - loss: 0.9828 - top_k_categorical_accuracy: 0.9243 - val_loss: 0.9558 - val_top_k_categorical_accuracy: 0.9262
Epoch 4/5
331200/331200 - 4s - loss: 0.9168 - top_k_categorical_accuracy: 0.9310 - val_loss: 0.9288 - val_top_k_categorical_accuracy: 0.9293
Epoch 5/5
331200/331200 - 4s - loss: 0.8721 - top_k_categorical_accuracy: 0.9357 - val_loss: 0.9070 - val_top_k_categorical_accuracy: 0.9332


In [48]:
#evaluate on unseen data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 93.14%


In [50]:
model.save('QuickDraw.h5')